In [7]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils import data
import torch.optim as optim
from matplotlib import pyplot as plt
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
from datetime import datetime
import pandas as pd

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
print(device)

nspl_input_ts = torch.load("/Users/palash.sethi/Downloads/Projects/Refinery Optimisation/NSPL/data/scaled_dataframes/nspl_ann/combined_ip.pt")
nspl_input_ts = nspl_input_ts.to(device)
nspl_output_ts = torch.load("/Users/palash.sethi/Downloads/Projects/Refinery Optimisation/NSPL/data/scaled_dataframes/nspl_ann/combined_op.pt")
nspl_output_ts = nspl_output_ts.to(device)

class ReactorSimulatorDataset(data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, x_path, y_path):
        'Initialization'
        self.x = torch.load(x_path)
        self.y = torch.load(y_path)
        # self.count = 0

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.x)

    def __getitem__(self, index):
        'Generates one sample of data'
        return self.x[index], self.y[index]

dataset = ReactorSimulatorDataset( \
        x_path = "/Users/palash.sethi/Downloads/Projects/Refinery Optimisation/NSPL/data/scaled_dataframes/nspl_ann/combined_ip.pt",
        y_path = "/Users/palash.sethi/Downloads/Projects/Refinery Optimisation/NSPL/data/scaled_dataframes/nspl_ann/combined_op.pt"
    )
batch_size = 16
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

training_generator = torch.utils.data.DataLoader(dataset, batch_size=batch_size,num_workers = 0 ,
                                           sampler=train_sampler)
validation_generator = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers = 0,
                                                sampler=valid_sampler)



False
cpu


In [9]:
def avg_loss(loss, size):
    return float(sum(loss)/size)

def MAPELoss(output, target):
    # print("lol: ", )
    # print("lol: ", output.shape)
    return torch.mean(torch.abs((target - output)) / (torch.abs(target) + torch.abs(output)))* 100


class Net2(torch.nn.Module):  # training only bottom stream
    def __init__(self):
        super(Net2, self).__init__()
        
        self.fc1 = torch.nn.Linear(34,80)
        self.activation1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(80,29)

    def forward(self, x):
        
        x = self.fc1(x)
        x = self.activation1(x)
        x = self.fc2(x)
        
        return x

net = Net2()
net.double()
net.to(device)
train_loss_list = []
val_loss_list = []
train_mape_loss_list = []
val_mape_loss_list = []
time_list = []
optimizer = optim.Adam(net.parameters(), lr=0.001)

epoch = 1000
for j in range(epoch):
    epoch_train_loss = []
    epoch_val_loss = []
    epoch_train_mape_loss = []
    epoch_val_mape_loss = []
    start_time = datetime.now()

    for x_batch, y_batch in training_generator:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        nn_output_batch = net(x_batch)
        loss = torch.mean(nn.MSELoss(reduction='none')(nn_output_batch, y_batch))
        epoch_train_loss.append(loss.item())
        mape_loss = MAPELoss(nn_output_batch, y_batch)
        epoch_train_mape_loss.append(mape_loss.item())
        loss.backward()
        optimizer.step()

    end_time = datetime.now()
    train_loss_list.append(avg_loss(epoch_train_loss, len(epoch_train_loss)))
    train_mape_loss_list.append(avg_loss(epoch_train_mape_loss, len(epoch_train_mape_loss)))
    time_list.append((end_time - start_time).microseconds)

    if not j % 10:
        for x_batch, y_batch in validation_generator:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            nn_output_batch = net(x_batch)
            loss = torch.mean(nn.MSELoss(reduction='none')(nn_output_batch, y_batch))
            epoch_val_loss.append(loss.item())
            epoch_val_mape_loss.append(MAPELoss(nn_output_batch, y_batch).item())
        val_loss_list.append(avg_loss(epoch_val_loss, len(epoch_val_loss)))
        val_mape_loss_list.append(avg_loss(epoch_val_mape_loss, len(epoch_val_mape_loss)))
        print("Training loss at epoch ", j,' is ', train_loss_list[-1])
        print("Training MAPE loss at epoch ", j,' is ', train_mape_loss_list[-1])
        print(("Validation loss at epoch ", j,' is ', val_loss_list[-1]))
        print(("Validation MAPE loss at epoch ", j,' is ', val_mape_loss_list[-1]))
        print(("Average Time Taken for epoch ", j,' is ', sum(time_list)/ len(time_list)))

Training loss at epoch  0  is  1.0164624020444746
Training MAPE loss at epoch  0  is  43.59238259352722
('Validation loss at epoch ', 0, ' is ', 0.6384130668042387)
('Validation MAPE loss at epoch ', 0, ' is ', 40.534754174929134)
('Average Time Taken for epoch ', 0, ' is ', 151550.0)
Training loss at epoch  10  is  0.3927425836744483
Training MAPE loss at epoch  10  is  36.17419031342816
('Validation loss at epoch ', 10, ' is ', 0.35643870872527944)
('Validation MAPE loss at epoch ', 10, ' is ', 35.488023623332104)
('Average Time Taken for epoch ', 10, ' is ', 255593.9090909091)


KeyboardInterrupt: 